In [72]:
!pip3 install -q kaggle
!pip3 install -q kaggle-cli
!pip3 install joblib
!pip3 install azureml
!pip3 install pip --upgrade
!pip3 install azureml.core
!pip3 install azureml-dataset-runtime --upgrade
!pip3 install azureml.widgets
!pip3 install azureml.train

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretsto

In [73]:
!pip3 install joblib

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!mkdir -p ~/.kaggle
!cp ~/sandbox/sandbox_env/Udacity/kaggle.json ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

{"username":"ocherif","key":"cb037f99cae382b7a67c68f8048e01be"}

In [3]:
#if not os.path.exists('/home/ocherif/sandbox/sandbox_env/Udacity/.kaggle'):
    #!mkdir ~/sandbox/sandbox_env/Udacity/.kaggle

In [70]:
import joblib
import argparse
import os
import kaggle
import numpy as np
import pandas as pd

from zipfile import ZipFile
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.core import Dataset, Datastore
#from azureml.data.datapath import DataPath


In [6]:
# Download dataset from kaggle
!~/.local/bin/kaggle datasets download -d gpreda/covid-world-vaccination-progress -p ./starter_file/kaggle/

covid-world-vaccination-progress.zip: Skipping, found more recently modified local copy (use --force to force download)


In [65]:
with ZipFile('./starter_file/kaggle/covid-world-vaccination-progress.zip', 'r') as datasetZip:
   # Extract all the contents of zip file in current directory
   datasetZip.extractall('./starter_file/kaggle')
    
path = "./starter_file/kaggle/country_vaccinations.csv"
#ds = TabularDatasetFactory.from_delimited_files(path=datastore_path, infer_column_types=True, separator=',', header=True, encoding='utf8')
run = Run.get_context()  

data = pd.read_csv("./starter_file/kaggle/country_vaccinations.csv").dropna()
def clean_data(data):
    # Clean and one hot encode data
    #x_df = data.to_pandas_dataframe().dropna()
    x_df = data
    # take the latest number of vaccinated people by country
    x_df['used_Vaccine'] = np.where(x_df.groupby('country')['total_vaccinations'].transform('max') > 0, True, False)
    y_df = x_df.pop("used_Vaccine").apply(lambda s: 1 if s == True else 0)
    
    countries = pd.get_dummies(x_df.country, prefix="country")
    iso_codes = pd.get_dummies(x_df.iso_code, prefix="iso_code")
    vaccines = pd.get_dummies(x_df.vaccines, prefix="vaccine")
    source_names = pd.get_dummies(x_df.source_name, prefix="source")
    #source_websites = pd.get_dummies(x_df.source_name, prefix="source_website")
    x_df.drop(["country","iso_code","vaccines","source_name","source_website"], inplace=True, axis=1)
    x_df = x_df.join([countries,iso_codes,vaccines,source_names])
    
    x_df['month']= pd.DatetimeIndex(x_df['date']).month
    x_df['date']=pd.to_datetime(x_df['date'], format='%Y-%m-%d')
    x_df['date']= pd.DatetimeIndex(x_df['date']).year
    
    return x_df,y_df

In [67]:
x, y = clean_data(data)

usage: ipykernel_launcher.py [-h] [--C C] [--max_iter MAX_ITER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/ocherif/.local/share/jupyter/runtime/kernel-2be056a2-d24f-47b3-8deb-067ea280679b.json


SystemExit: 2

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
